In [1]:
import os
import sys
os.environ["SPARK_HOME"] = "/usr/lib/spark/"
os.environ["PYSPARK_PYTHON"]= "/home/cloudera/anaconda2/bin/python"
os.environ["PYSPARK_DRIVER_PYTHON"]= "/home/cloudera/anaconda2/bin/python"
os.environ["SPARK_YARN_USER_ENV"]= "/home/cloudera/anaconda2/bin/python"

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.8.1-src.zip'))
execfile(os.path.join(spark_home, 'python/pyspark/shell.py'))

from pyspark import SparkContext, SparkConf

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.6.0
      /_/

Using Python version 2.7.13 (default, Dec 20 2016 23:09:15)
SparkContext available as sc, HiveContext available as sqlContext.


In [2]:
x= sc.textFile("/user/cloudera/kaggle/orders").map(lambda x: x.split(','))

In [3]:
test = x.filter(lambda x: x[2]=='test').map(lambda x: x[1]).distinct()

In [4]:
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

In [9]:
model = MatrixFactorizationModel.load(sc, "/user/cloudera/kaggle/model")

In [10]:
model.recommendProducts(1,10)

[Rating(user=1, product=13176, rating=0.6833920201082382),
 Rating(user=1, product=12341, rating=0.2606711224255599),
 Rating(user=1, product=6184, rating=0.25304948680000483),
 Rating(user=1, product=43352, rating=0.23523993177616165),
 Rating(user=1, product=21903, rating=0.21333270362711204),
 Rating(user=1, product=21137, rating=0.1810016485532501),
 Rating(user=1, product=196, rating=0.16125479473361332),
 Rating(user=1, product=42265, rating=0.1545010261115591),
 Rating(user=1, product=35561, rating=0.1408376275833716),
 Rating(user=1, product=39275, rating=0.13990913621452966)]

In [21]:
joined_prior =  sc.textFile("/user/cloudera/kaggle/order_product_prior_joined")

In [34]:
j= joined_prior.map(lambda x : x.split(',')).map(lambda x: (x[1], x[7])).map(lambda x : (x[0], len(x[1].split(' ')))).\
    reduceByKey(lambda x, y : x+y).map(lambda x : x[1]).collect()

In [35]:
print reduce(lambda x,y : x+y, j )/len(j)

157


In [11]:
test_list = test.map(lambda x : int(x)).collect()

In [1]:
import time
def recommend(list):
    start_time = time.time()
    i=0
    x= []
    for val in test_list:
        if (i%10 == 0):
            print "recommendor count has crossed %d and time lapsed %d secs" %(i, time.time() - start_time)
        x.append(model.recommendProducts(val,2))
        if(i>30):
            break
        i= i+1
    return x

In [25]:
x= recommend(test_list)

recommendor count has crossed 0 and lime lapsed 0 secs


recommendor count has crossed 10 and lime lapsed 44 secs


recommendor count has crossed 20 and lime lapsed 61 secs


recommendor count has crossed 30 and lime lapsed 76 secs


##### it seems like for 10 prediction approximate 20 seconds of time is required 
##### this implies 2 sec per user hence for 75000 rows of test set it might take 150000 secs 
##### This might require approx two day to complete that is not possible with the personal machine 
##### Only thing is that once we have the list of predictions we can convert it to rdd and set it into right format of the output.

##### Since the recommemdProduct function takes a lot of time we may think of other methods to predict 
##### Lets take in predict_all function to find the output rating of user and products of interest.
##### All we need is to find out the right combination of user and product to get the rating.
##### As cartegion multiplication of the all products with all test users will lead to large data set.
##### so product which user have purchased earlier can be used or can use the aisles that user have considered with rating for all products under the aisles 
##### we can use aisles data set as broadcast variable and limit the no of product to get the rating for the users  

In [5]:
aisles = sc.textFile("file:///home/cloudera/Desktop/kaggle/aisles.csv")

In [4]:
order_prod_prio_inter = sc.textFile("/user/cloudera/kaggle/order_prior_inter").map(lambda x: x.split(','))

In [5]:
test_users = test.collect()

In [11]:
test_user_prod.persist()

PythonRDD[15] at RDD at PythonRDD.scala:43

In [12]:
test_user_prod.count()